####Download Video form Youtube

In [ ]:
#Library for downloading youtube vid 
!pip install pytube 

In [ ]:
from pytube import YouTube 

In [ ]:
youtube_video = "https://www.youtube.com/watch?v=5-rCKo4CBgM"

In [ ]:
yt = YouTube(youtube_video)

In [ ]:
#Download audio from youtube
#filtering only mp4 (.filter)
#return objek yang memenuhi kondisi (.first)

yt.streams\
    .filter(only_audio = True, file_extension = 'mp4')\
    .first()\
    .download(filename ='audio.mp4')

In [ ]:
import IPython.display as display

In [ ]:
path = "/content/audio.mp4"

In [ ]:
display.Audio(path, autoplay=True)

In [ ]:
#convert mp4 to wav
! ffmpeg -i audio.mp4 -acodec pcm_s16le -ar 16000 audio.wav

####ASR 

In [ ]:
!pip install torch --upgrade

In [ ]:
!pip install huggingsound --upgrade

In [ ]:
from huggingsound import SpeechRecognitionModel
import torch

In [ ]:
#Checking GPU if CUDA enabled -> output cuda, if not output will be cpu
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [ ]:
#pretrained model using wav2vec2 for
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device=device)

####Audio Slicing

In [ ]:
import librosa
import soundfile as sf

In [ ]:
input_file ='/content/audio.wav'

In [ ]:
# to retrieve the sampling rate of an audio file
print(librosa.get_samplerate(input_file)) 

stream = librosa.stream(
    input_file,
    block_length=30, #Slicing duration to 30 seconds
    frame_length=16000, #length of each audio frame per block
    hop_length=16000 #overlap between gap, set to 16000 means that there is no overlap
)

In [ ]:
#Save slicing result from "stream"
for i, speech in enumerate(stream):
  sf.write(f'{i}.wav', speech, 16000) #to write audio data to a file

In [ ]:
i

####Audio Transcript (Speech to Text)

In [ ]:
#Generate list of audio 
audio_path = []
for a in range(i+1):
  audio_path.append(f'/content/{a}.wav')

In [ ]:
audio_path

In [ ]:
#transcript audio_path and stored in the transcriptions 
transcriptions = model.transcribe(audio_path)

In [ ]:
full_transcript = ""

In [ ]:
#combine all transcript in full_transcript
for i in transcriptions :
  full_transcript += ''.join(i['transcription'])

In [ ]:
#length of characters 
len(full_transcript)

In [ ]:
import pprint

In [ ]:
pprint.pprint(full_transcript)

In [ ]:
#Save full_transcript to txt file 
transcript_file = "/content/transcript.txt"

#open file 
with open(transcript_file, "w") as file:
  file.write(full_transcript)

print("Transcript saved to", transcript_file)

In [ ]:
with open("transcript.txt", "r") as file:
  trans = file.read().replace('\n', '')
trans = trans.replace("\ufeff", "")

####Text Summarizer

In [ ]:
!pip install --upgrade transformers

In [ ]:
#pipeline library in transformers, used for summaryzation
from transformers import pipeline

In [ ]:
#pretrained model for summarization using bart using CNN daily mail dataset 
bart_model ="facebook/bart-large-cnn"
summarization = pipeline('summarization', model=bart_model)

In [ ]:
# summarized_text = summarization(full_transcript)

In [ ]:
# summarized_text[0]['summary_text']

In [ ]:
#summarize per 500 characters 
num_iters = int(len(full_transcript)/500)
summarized_text = []
for i in range(0, num_iters + 1):
  start = 0
  start = i * 500
  end = (i + 1) * 500
  print("INPUT TEXT")
  pprint.pprint(full_transcript[start:end])
  out = summarization(full_transcript[start:end], min_length = 5, max_length=50)
  out = out[0]
  out = out['summary_text']
  print("SUMMARY TEXT")
  pprint.pprint(out)
  summarized_text.append(out)

In [ ]:
len(str(summarized_text))

In [ ]:
summary = " ".join(summarized_text)
pprint.pprint(summary)

In [ ]:
#save summary result to txt file 
result = "/content/summary_result.txt"

with open(result, "w") as file:
  file.write(summary)

print("Summary Result saved to", result)

##Evaluation 

Matrix evaluation using ROUGE for BART pretrained model 

Doing evaluation compared summarization using BART model and human summarization

In [ ]:
#Summary result without pretrained model -> only using pipeline from transformer 
with open("summary_Pip.txt", 'r') as file:
  data1 = file.read().replace('\n', '')
data1 = data1.replace("\ufeff","")

In [ ]:
#Summary result using BART model 
with open("summary_result.txt", 'r') as file:
  data2 = file.read().replace('\n', '')
data2 = data2.replace("\ufeff","")

In [ ]:
!pip install rouge 

In [ ]:
from rouge import Rouge

In [ ]:
rouge1 = Rouge()

In [ ]:
score = rouge1.get_scores(data1, data2, avg=True)

In [ ]:
print(score)

Matrix Evaluation using WER for wav2vec2 pretrained model 

Compared transcript result using API and wav2vec2

In [ ]:
import nltk

In [ ]:
#Transcript result using youtube api
with open("transcript_api.txt", 'r') as file:
  data3 = file.read().replace('\n', '')
data3 = data1.replace("\ufeff","")

In [ ]:
#Transcript result using wav2vec2
with open("transcript.txt", 'r') as file:
  data4 = file.read().replace('\n', '')
data4 = data1.replace("\ufeff","")

In [ ]:
def calculate_wer(data3, data4):
  with open("transcript_api.txt", 'r') as file:
    data3 = file.read().strip().lower().split()

  with open("transcript.txt", 'r') as file:
    data4 = file.read().strip().lower().split()

  wer = nltk.edit_distance(data3, data4) / len(data3)
  return wer

In [ ]:
ref = "transcript_api.txt"
hyp = "transcript.txt"

In [ ]:
score = calculate_wer(ref, hyp)

In [ ]:
print("WER Score :", score)